# MADE. Advanced ML. Home assignment #3

## The Adventure of the Dancing Men

<img src="./data/img/dancing_men.png" width="600" align="left">

### 1.
Реализуйте базовый частотный метод по Шерлоку Холмсy

- подсчитайте частоты букв по корпусам (пунктуацию и капитализацию можно просто опустить, а вот пробелы лучше оставить);
- возьмите какие-нибудь тестовые тексты (нужно взять по меньшей мере 2-3 предложения, иначе вряд ли сработает), зашифруйте их посредством случайной перестановки символов;
- расшифруйте их таким частотным методом.

In [211]:
from collections import Counter
from itertools import product
import os
import random
import re
from typing import Dict, Optional, List

import matplotlib.pyplot as plt; plt.ion()
import numpy as np
from tqdm.notebook import tqdm

In [18]:
CORPUS_DATAPATH = "data/corpora"
WAR_AND_PEACE_RU_FILEPATH = os.path.join(CORPUS_DATAPATH, "WarAndPeace.txt")
WAR_AND_PEACE_EN_FILEPATH = os.path.join(CORPUS_DATAPATH, "WarAndPeaceEng.txt")
ANNA_KARENINA_RU_FILEPATH = os.path.join(CORPUS_DATAPATH, "AnnaKarenina.txt")

In [28]:
def read_text(filepath: str) -> str:
    with open(filepath, 'r') as f:
        text = f.read()
        text = text.replace('\n', ' ')
    return text

In [83]:
def preprocess_text(text: str, languare: str = 'ru') -> str:
    """Process text with following steps:
    1. To lower case
    2. Take only letters from appropriate languare
    3. Replace sequence of subspaces to one
    """
    processed = text.lower()
    if languare == "ru":
        pattern_to_delete = re.compile("[^а-яё ]")
    elif languare == "en":
        pattern_to_delete = re.compile("[^a-z ]")
    else:
        raise ValueError(f"Unknown languale: {languare}")
    processed = re.sub(pattern_to_delete, " ", processed)
    processed = re.sub(r"\s+", " ", processed)
    return processed

In [84]:
CORPUS = dict()

for text_id, text_filepath, lang in tqdm(zip(
    ["war_and_peace_ru", "war_and_peace_en", "anna_karenina_ru"],
    [WAR_AND_PEACE_RU_FILEPATH, WAR_AND_PEACE_EN_FILEPATH, ANNA_KARENINA_RU_FILEPATH],
    ["ru", "en", "ru"]
)):
    text_i = read_text(text_filepath)
    CORPUS[text_id] = preprocess_text(text_i, lang)

In [86]:
for text_id, text_processed in CORPUS.items():
    print(text_id)
    print(text_processed[:100], "\n")

war_and_peace_ru
 война и мир самый известный роман льва николаевича толстого как никакое другое произведение писател 

war_and_peace_en
 the project gutenberg ebook of war and peace by leo tolstoy this ebook is for the use of anyone any 

anna_karenina_ru
 анна каренина один из самых знаменитых романов льва толстого начинается ставшей афоризмом фразой вс 



- Частоты по корпусам

In [92]:
def calculate_letter_frequencies(text: str) -> Dict[str, float]:
    """Calculate relative frequencies of characters from text"""
    letter_counter = Counter(text)
    letter_counter = {i: j / len(text) for i, j in letter_counter.items()}
    letter_counter = dict(
        sorted(
            letter_counter.items(),
            key=lambda x: x[1],
            reverse=True
        )
    )
    return letter_counter

- Зашифруем и попробуем расшифровать тестовые данные случайной перестановкой символов

In [222]:
def get_alphabet(language: str) -> List[str]:
    """Generate list of letters for appripriate languare"""
    if language == "ru":
        alphabet = [chr(i) for i in range(ord('а'), ord('я') + 1)] + ['ё', " "]
    elif language == "en":
        alphabet = [chr(i) for i in range(ord('a'), ord('z') + 1)] + [" "]
    else:
        raise ValueError(f"Unknown languale: {languare}")
    return alphabet


def get_shuffled_encoder(language: str = "ru") -> Dict[str, str]:
    """Provide mapping from original alphabet to shuffled one"""
    mapping = dict()
    alphabet = get_alphabet(language)
    new_alphabet = alphabet.copy()
    random.shuffle(new_alphabet)
    return dict(zip(alphabet, new_alphabet))


def encode_text(text: str, encoder: Dict[str, str]) -> str:
    """Encode text based on encoder latters mapping"""
    result = ""
    for i in text:
        result += encoder.get(i, "?")
    return result


def decode_text(
    encoded_text: str,
    train_freq: Dict[str, str]
) -> str:
    """Decode text using letter frequencies previously calculated"""
    test_freq = calculate_letter_frequencies(encoded_text)
    
    decoder = dict()
    test_letters = list(test_freq.keys())
    train_letters = list(train_freq.keys())
    for i, j in enumerate(test_letters):
        if i < len(train_letters):
            decoder[j] = train_letters[i]
        else:
            decoder[j] = "?"
    
    result = encode_text(encoded_text, decoder)
    return result


def calculate_accuracy(text_true, text_pred):
    """Accuracy score of exact letters matching"""
    assert len(text_true) == len(text_pred), "Text lengths should match exactly"
    return sum([i == j for i, j in zip(text_true, text_pred)]) / len(text_true)

In [215]:
random_encoder = get_shuffled_encoder("ru")

In [216]:
war_and_peace_ru_freq = calculate_letter_frequencies(CORPUS['war_and_peace_ru'])

In [217]:
# Пример из Train dataset-а
example_from_train = CORPUS['war_and_peace_ru'][:51]
example_from_train_encoded = encode_text(example_from_train, random_encoder)
example_from_train_decoded = decode_text(
    CORPUS['war_and_peace_ru'],
    train_freq=war_and_peace_ru_freq
)[:51]

print("Train Example:", example_from_train)
print("Encoded:", example_from_train_encoded)
print("Decoded:", example_from_train_decoded)
print(f"Example accuracy: {calculate_accuracy(example_from_train, example_from_train_decoded)}")

Train Example:  война и мир самый известный роман льва николаевича
Encoded: гьджсщгчгъчлгвщъяжгчфьтв сяжглдъщсгюпьщгсчцдющтьчущ
Decoded:  война и мир самый известный роман льва николаевича
Example accuracy: 1.0


In [218]:
# Пример из Test dataset-а, где частоты посчитаны на целом тексте:
example_from_test = CORPUS['anna_karenina_ru'][:80]
example_from_test_encoded = encode_text(example_from_test, random_encoder)
example_from_test_decoded = decode_text(
    CORPUS['anna_karenina_ru'],
    train_freq=war_and_peace_ru_freq
)[:80]

print("Test Example:", example_from_test)
print("Encoded:", example_from_test_encoded)
print("Decoded:", example_from_test_decoded)
print(f"Example accuracy: {calculate_accuracy(example_from_test, example_from_test_decoded)}")

Test Example:  анна каренина один из самых знаменитых романов льва толстого начинается ставшей
Encoded: гщссщгцщлтсчсщгдычсгчфгвщъянгфсщътсч янглдъщсдьгюпьщг дюв додгсщучсщт вкгв щьзтж
Decoded:  еиие кераиние одни нч семьх чиемаинтьх ромеиов лгве толстоыо иебниеатся стевшаж
Example accuracy: 0.525


### 2.

Вряд ли в результате получилась такая уж хорошая расшифровка, разве что если вы брали в качестве тестовых данных целые рассказы. Но и Шерлок Холмс был не так уж прост: после буквы E, которая действительно выделяется частотой, дальше он анализировал уже конкретные слова и пытался угадать, какими они могли бы быть. Я не знаю, как запрограммировать такой интуитивный анализ, так что давайте просто сделаем следующий логический шаг:

- подсчитайте частоты биграмм (т.е. пар последовательных букв) по корпусам;
- проведите тестирование аналогично п.1, но при помощи биграмм

In [203]:
def calculate_bigram_frequencies(text: str) -> Dict[str, float]:
    """Calculate relative frequencies of bigrams from text"""
    bigram_counter = Counter()
    for i in range(len(text) - 1):
        bigram_counter[text[i: i + 2]] += i
    
    n_bigrams = sum(bigram_counter.values())
    bigram_counter = dict(
        sorted(
            bigram_counter.items(),
            key=lambda x: x[1],
            reverse=True
        )
    )
    bigram_counter = {i: j / n_bigrams for i, j in bigram_counter.items()}
    return bigram_counter

In [240]:
# abc -> ab bc
# ab bc -> abc

In [259]:
# anna_karenina_encoded = encode_text(CORPUS['anna_karenina_ru'], random_encoder)
# anna_karenina_encoded_bigrams_freq = calculate_bigram_frequencies(anna_karenina_encoded)

In [261]:
# war_and_peace_ru_bigram_freq = calculate_bigram_frequencies(CORPUS['war_and_peace_ru'])

In [267]:
# for i in range(len(anna_karenina_encoded) - 1):
#     bigram_i = anna_karenina_encoded[i: i + 2]
#     break

In [278]:
# bigram_decoder = dict(zip(list(anna_karenina_encoded_bigrams_freq), list(war_and_peace_ru_bigram_freq)))

In [291]:
# i = 100
# print(anna_karenina_encoded[i: i + 2], anna_karenina_encoded[i + 1: i + 3], '-->', end=' ')
# print(
#     bigram_decoder.get(anna_karenina_encoded[i: i + 2], '??'),
#     bigram_decoder.get(anna_karenina_encoded[i + 1: i + 3], '??')
# )

тг гв --> и   п


In [298]:
# alphabet_decoder_from_bigrams = dict()
# for i, j in bigram_decoder.items():
#     for a, b in zip(i, j):
#         if a in alphabet_decoder_from_bigrams:
#             alphabet_decoder_from_bigrams[a].append(b)
#         else:
#             alphabet_decoder_from_bigrams[a] = [b]

In [339]:
# alphabet_decoder_from_bigrams_result = dict()
# not_assigned = list()
# for i, j in alphabet_decoder_from_bigrams.items():
#     j_updated = [k for k in j if k not in alphabet_decoder_from_bigrams_result.values()]
#     if len(j_updated) == 0:
#         not_assigned.append(i)
#         continue
#     alphabet_decoder_from_bigrams_result[i] = Counter(j_updated).most_common(1)[0][0]
    
# alphabet_decoder_from_bigrams_result.update(
#     dict(zip(
#         not_assigned,
#         [i for i in get_alphabet("ru") if i not in alphabet_decoder_from_bigrams_result.values()]
#     ))
# )

In [341]:
# encode_text(anna_karenina_encoded, alphabet_decoder_from_bigrams_result)[:100]

' тсст ктнесаст охас аз итгыш зстгесауыш ногтсов юмвт уоюиуоро стчастеуид иутвяеэ тщоназгог щнтзоэ ви'

### 3.

Но и это ещё не всё: биграммы скорее всего тоже далеко не всегда работают. Основная часть задания — в том, как можно их улучшить:
- предложите метод обучения перестановки символов в этом задании, основанный на MCMC-сэмплировании, но по-прежнему работающий на основе статистики биграмм;
- реализуйте и протестируйте его, убедитесь, что результаты улучшились.

### 4.

Расшифруйте сообщение:

←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸ ↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊ ↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒ ↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝← ⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏

Или это (они одинаковые, второй вариант просто на случай проблем с юникодом):

დჳჵჂႨშႼႨშჂხჂჲდႨსႹႭ􏰀ႣჵისႼჰႨჂჵჂႨႲႹႧჲჂႨსႹႭ􏰀ႣჵისႼჰႨჲდႩჳჲႨ􏰀ႨႠჲႹქႹႨჳႹႹჱჶდსჂႽႨႩႹჲႹႭႼჰႨჵდქႩႹႨ ႲႭႹႧჂჲႣჲიႨჳႩႹႭდდႨშჳდქႹႨშႼႨშჳდႨჳხდჵႣჵჂႨႲႭႣშჂჵისႹႨჂႨႲႹჵ􏰀ႧჂჲდႨ􏰀ႣႩჳჂ􏰀ႣჵისႼჰႨჱႣჵჵႨეႣႨႲႹჳჵდხს დდႨႧდჲშდႭჲႹდႨეႣხႣსჂდႨႩ􏰀ႭჳႣႨႾႹჲႽႨႩႹსდႧსႹႨႽႨსჂႧდქႹႨსდႨႹჱდჶႣნ

### 5.

Бонус: а что если от биграмм перейти к триграммам (тройкам букв) или даже больше? Улучшатся ли результаты? Когда улучшатся, а когда нет? Чтобы ответить на этот вопрос эмпирически, уже может понадобиться погенерировать много тестовых перестановок и последить за метриками, глазами может быть и не видно.

### 6.

Бонус: какие вы можете придумать применения для этой модели? Пляшущие человечки ведь не так часто встречаются в жизни (хотя встречаются! и это самое потрясающее во всей этой истории, но об этом я расскажу потом).